In [1]:
import os
import random

import numpy as np
import pandas
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic", #"NanumGothicCoding",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
bus_info = pd.read_excel('../../data/서울교통수단이동/master/시내버스 2022.11월 인가현황.xlsx')
bus_info.columns = list(map(lambda x: x.replace('\n', ''), bus_info.columns))

# 맞춤버스
bus_fit = set(map(str, bus_info.loc[bus_info['유형'] == '맞춤', '노선번호']))

In [4]:
route_master = pd.read_csv('../../data/서울교통수단이동/202209/TR202100010001_00/TBIS_MS_ROUTE.csv')
station_master = pd.read_csv('../../data/서울교통수단이동/202209/TR202100010001_00/TBIS_MS_STTN.csv', on_bad_lines='skip')
route_node_master = pd.read_csv('../../data/서울교통수단이동/202209/TR202100010001_00/TBIS_MS_ROUTE_NODE.csv')

In [3]:
def to_trip_velo(route_master, route_node_master, bus_fit, trip_time, trip_turn):
    trip_time = trip_time.loc[:, ['ROUTE_ID', 'FROM_STTN_SN', 'TO_STTN_SN', 'FROM_STTN_ID', 'TO_STTN_ID', 'TRIP_TIME_18H', 'TRIP_TIME_19H', 'TRIP_TIME_20H', 'TRIP_TIME_21H']]
    trip_time = trip_time.loc[~((trip_time['TRIP_TIME_18H'] == 0) & (trip_time['TRIP_TIME_19H'] == 0) & (trip_time['TRIP_TIME_20H'] == 0) & (trip_time['TRIP_TIME_21H'] == 0))]

    trip_turn = trip_turn.loc[:, ['ROUTE_ID', 'STTN_SN', 'STTN_ID', 'CNT_18H', 'CNT_19H', 'CNT_20H', 'CNT_21H']]
    trip_turn.fillna(0.0, inplace=True)

    # 노선마스터와 합치기
    trip_time = pd.merge(trip_time, route_master, on='ROUTE_ID')
    trip_time.drop(columns=['DSTNC'], inplace=True)

    # 필요 없는 버스 제거
    trip_time = trip_time.loc[trip_time['ROUTE_TY'].isin(['간선', '지선'])]  # 간선, 지선 버스만
    trip_time = trip_time.loc[trip_time['ROUTE_NM'].str[0] != 'N']         # 야간버스 제거
    trip_time = trip_time.loc[~trip_time['ROUTE_NM'].isin(bus_fit)]        # 맞춤버스 제거

    # merge trip_time, trip_trun
    trip_speed = pd.merge(trip_time, trip_turn, left_on=['ROUTE_ID', 'TO_STTN_ID', 'TO_STTN_SN'], right_on=['ROUTE_ID', 'STTN_ID', 'STTN_SN'])
    trip_speed.drop(columns=['STTN_ID', 'STTN_SN'], inplace=True)

    # merge trip_speed, route_node_master
    trip_speed = pd.merge(trip_speed, route_node_master, left_on=['ROUTE_ID', 'TO_STTN_ID', 'TO_STTN_SN'], right_on=['ROUTE_ID', 'STTN_ID', 'STTN_SN'])
    trip_speed.drop(columns=['STTN_ID', 'STTN_SN'], inplace=True)

    # 총합 = 평균 * cnt
    avg = trip_speed.loc[:, ['TRIP_TIME_18H', 'TRIP_TIME_19H', 'TRIP_TIME_20H', 'TRIP_TIME_21H']].values
    count = trip_speed.loc[:, ['CNT_18H', 'CNT_19H', 'CNT_20H', 'CNT_21H']].values

    time_sum = avg * count

    # 18 ~ 21시 평균속도
    trip_speed['TRIP_TIME_18_21'] = time_sum.sum(axis=1) / count.sum(axis=1)
    trip_speed['TRIP_VELO_18_21'] = trip_speed['LINK_DSTNC_ACMTL'] / trip_speed['TRIP_TIME_18_21'] * 3600 / 1000
    trip_speed.dropna(inplace=True)

    return trip_speed.loc[:, ['ROUTE_ID', 'FROM_STTN_SN', 'TO_STTN_SN', 'FROM_STTN_ID', 'TO_STTN_ID', 'TRIP_VELO_18_21']]

In [5]:
route_143_id = route_master.loc[route_master['ROUTE_NM'] == '143', 'ROUTE_ID'].values[0]

In [6]:
date = (('2022', '09'), ('2022', '10'), ('2022', '11'), ('2022', '12'), ('2023', '01'), ('2023', '02'))

trip_velo = pd.DataFrame(columns=['ROUTE_ID', 'FROM_STTN_SN', 'TO_STTN_SN', 'FROM_STTN_ID', 'TO_STTN_ID', 'TRIP_VELO_18_21'])

for year, month in date:
    path = f'../../data/서울교통수단이동/{year}{month}'
    dirs = os.listdir(path)
    print('year, month:', year, month)

    for dir_ in dirs:
        print('    dir:', dir_)
        route_master = pd.read_csv(f'{path}/{dir_}/TBIS_MS_ROUTE.csv')
        route_node_master = pd.read_csv(f'{path}/{dir_}/TBIS_MS_ROUTE_NODE.csv')
        trip_time = pd.read_csv(f'{path}/{dir_}/TPSS_ROUTE_SECTION_SPEED_H.csv')
        trip_turn = pd.read_csv(f'{path}/{dir_}/TPSS_STA_ROUTE_INFO_H_TURN.csv')

        trip_velo_temp = to_trip_velo(route_master, route_node_master, bus_fit, trip_time, trip_turn)
        trip_velo_temp = trip_velo_temp.loc[trip_velo_temp['ROUTE_ID'] == route_143_id]

        trip_velo = pd.concat([trip_velo, trip_velo_temp])

trip_velo

year, month: 2022 09
    dir: TR202100010001_00
    dir: TR202100010001_01
    dir: TR202100010001_02
    dir: TR202100010001_03
    dir: TR202100010001_04
    dir: TR202100010001_05
    dir: TR202100010001_06
    dir: TR202100010001_07
    dir: TR202100010001_08
    dir: TR202100010001_09
    dir: TR202100010001_10
    dir: TR202100010001_11
    dir: TR202100010001_12
    dir: TR202100010001_13
    dir: TR202100010001_14
    dir: TR202100010001_15
    dir: TR202100010001_16
    dir: TR202100010001_17
    dir: TR202100010001_18
    dir: TR202100010001_19
    dir: TR202100010001_20
    dir: TR202100010001_21
    dir: TR202100010001_22
    dir: TR202100010001_23
    dir: TR202100010001_24
    dir: TR202100010001_25
    dir: TR202100010001_26
    dir: TR202100010001_27
year, month: 2022 10
    dir: TR202100010001_00
    dir: TR202100010001_01
    dir: TR202100010001_02
    dir: TR202100010001_03
    dir: TR202100010001_04
    dir: TR202100010001_05
    dir: TR202100010001_06
    dir: TR20

,ROUTE_ID,FROM_STTN_SN,TO_STTN_SN,FROM_STTN_ID,TO_STTN_ID,TRIP_VELO_18_21
23407,100100022,99,100,100000108,100000123,10.091553
23408,100100022,7,8,107000029,107000031,14.470073
23409,100100022,62,63,122000136,122000130,11.097445
23410,100100022,57,58,122000241,122000242,19.538028
23411,100100022,66,67,122000145,122000139,13.245913
...,...,...,...,...,...,...
3766,100100022,23,24,102000057,102000060,34.360807
3767,100100022,79,80,121000129,121000131,10.626417
3768,100100022,45,46,122000330,122000146,16.706682
3769,100100022,33,34,121000101,122000001,14.506667


In [7]:
trip_velo_mean = trip_velo.groupby(by=['ROUTE_ID', 'FROM_STTN_SN', 'TO_STTN_SN', 'FROM_STTN_ID', 'TO_STTN_ID'], as_index=False)['TRIP_VELO_18_21'].mean()
trip_velo_mean

,ROUTE_ID,FROM_STTN_SN,TO_STTN_SN,FROM_STTN_ID,TO_STTN_ID,TRIP_VELO_18_21
0,100100022,1,2,107000071,107000073,22.759617
1,100100022,2,3,107000073,107000518,9.662588
2,100100022,3,4,107000518,107000075,8.569898
3,100100022,4,5,107000075,107000077,19.238030
4,100100022,5,6,107000077,107000079,12.787229
...,...,...,...,...,...,...
170,100100022,111,112,107000078,107000076,16.701709
171,100100022,112,113,107000076,107000519,6.497895
172,100100022,113,114,107000519,107000074,10.469869
173,100100022,114,115,107000074,107000072,20.215517


In [ ]:
trip_velo_mean